<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pegar-links-das-páginas-de-bairros" data-toc-modified-id="Pegar-links-das-páginas-de-bairros-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pegar links das páginas de bairros</a></span></li><li><span><a href="#Iterar-pelos-links,-buscando-informações-demográficas" data-toc-modified-id="Iterar-pelos-links,-buscando-informações-demográficas-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Iterar pelos links, buscando informações demográficas</a></span></li><li><span><a href="#Salvar-resultado" data-toc-modified-id="Salvar-resultado-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Salvar resultado</a></span></li></ul></div>

In [1]:
import sys,os,re,urllib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from bs4 import BeautifulSoup

In [2]:
sys.path.append('..')

In [3]:
from config import *

# Pegar links das páginas de bairros

In [5]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_bairros_da_cidade_do_Rio_de_Janeiro'

page = urllib.request.urlopen(url)
soup = BeautifulSoup(page,'lxml')
table = soup.find('table',class_='wikitable')
table_str = str(table)
pat = r'(href=")(/wiki/)(([A-Z]|[a-z]|[0-9]|%|_|\(|\)|/)+)(" title)'
links = ['https://pt.wikipedia.org/wiki/{}'.format(i[2]) for i in re.findall(pat,table_str)]

# Iterar pelos links, buscando informações demográficas

In [6]:
def buscar_area(link_table):
    pat = '((Área total<\/a>\\n<\/td>\\n<td style="vertical-align: top; text-align: left;">)([0-9]|,)+)'
    try:
        area = float(re.findall(pat,str(link_table))[0][0].split('>')[-1].replace(',','.'))
        return area
    except:
        return None
    
def buscar_pop(link_table):
    pat = '((População total</a>\n</td>\n<td style="vertical-align: top; text-align: left;">)([0-9]|,)+)'
    try:
        pop = float(re.findall(pat,str(link_table))[0][0].split('>')[-1].replace(',','.'))
        return pop
    except:
        return None
    
def buscar_idh(link_table):
    pat = '((IDH</a>\n</td>\n<td style="vertical-align: top; text-align: left;">(<span style="color: #([0-9]|[A-Z]|;)+">)?(<font color="#([0-9]|[A-Z]|;)+">)?)([0-9]|,)+)'
    try:
        idh = float(re.findall(pat,str(link_table))[0][0].split('>')[-1].replace(',','.'))
        return idh
    except:
        return None

In [15]:
results = []
for link in tqdm(links[1:]):
    page = urllib.request.urlopen(link)
    soup = BeautifulSoup(page,'lxml')
    
    tables = soup.find_all('table')
    if len(tables) > 0:
        
        for t in soup.find_all('table'):
            if 'População' in str(t):
                link_table = t
                break

        area = buscar_area(link_table)
        pop  = buscar_pop(link_table)
        idh  = buscar_idh(link_table)
        
    else:
        area,pop,idh = None,None,None

    results.append((link,area,pop,idh))

# Salvar resultado

In [16]:
df = pd.DataFrame(results,columns=['link','area','populacao','idh'])

In [19]:
df

,link,area,populacao,idh
0,https://pt.wikipedia.org/wiki/S%C3%A3o_Crist%C...,410.56,26.510,0.833
1,https://pt.wikipedia.org/wiki/Benfica_(bairro_...,173.64,25.081,0.825
2,https://pt.wikipedia.org/wiki/Caju_(bairro_do_...,534.75,20.477,0.753
3,https://pt.wikipedia.org/wiki/Catumbi,53.95,12.000,0.802
4,https://pt.wikipedia.org/wiki/Centro_(Rio_de_J...,572.31,41.142,0.894
...,...,...,...,...
171,https://pt.wikipedia.org/wiki/Vicente_de_Carva...,211.69,29.000,0.807
172,https://pt.wikipedia.org/wiki/Vig%C3%A1rio_Geral,211.69,29.000,0.807
173,https://pt.wikipedia.org/wiki/Vila_da_Penha,143.57,NaN,0.909
174,https://pt.wikipedia.org/wiki/Vila_Kosmos,143.57,NaN,0.909


In [20]:
df.to_pickle(os.path.join(TRTD_DATA_PATH,'wiki_demo_data.pkl'))
df.to_csv(os.path.join(TRTD_DATA_PATH,'wiki_demo_data.csv'))